In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

from hyperlib.manifold.lorentz import Lorentz
from hyperlib.manifold.poincare import Poincare
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from spektral.data import SingleLoader
from spektral.datasets.citation import Citation
from spektral.transforms import LayerPreprocess
from spektral.layers import GCNConv
from spektral.models.gcn import GCN

In [2]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [3]:
import hyperlib

In [4]:
from hyperlib.nn.layers.linear import ActivationHyperbolic, LinearHyperbolic
from hyperlib.nn.layers.graph import HGCLayer, HyperbolicAggregation, HGCNLP


In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
#ConvHyperbolic(16, self.manifold, self.c0, self.c1, activation="relu")
hgc_layer = HGCLayer(100, Poincare(), 0.4, "relu")

In [7]:
from hyperlib.utils.data_utils import load_data, load_data_lp, mask_edges, process

## Load data

In [8]:
from spektral.datasets import citation

dataset = citation.Cora()
graph = dataset[0]

# Node features
X = graph.x 


D:\PythonVirtualEnvironments\hyperlib\lib\site-packages\scipy\sparse\_index.py:145: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [9]:
graph.n_nodes

2708

In [10]:
graph.a

<2708x2708 sparse array of type '<class 'numpy.float32'>'
	with 10556 stored elements in Compressed Sparse Row format>

In [11]:
X.shape

(2708, 1433)

In [12]:
data = load_data_lp("cora", True, "resources\\data\\hgcn\\cora")

val_prop = 0.1
test_prop = 0.2
split_seed = 42
normalize_adj = False
normalize_feats = False

adj = data['adj_train']

adj_train, train_edges, train_edges_false, val_edges, val_edges_false, test_edges, test_edges_false = mask_edges(
        adj, val_prop, test_prop, split_seed
)
data['adj_train'] = adj_train
data['train_edges'], data['train_edges_false'] = train_edges, train_edges_false
data['val_edges'], data['val_edges_false'] = val_edges, val_edges_false
data['test_edges'], data['test_edges_false'] = test_edges, test_edges_false
    
data['adj_train_norm'], data['features'] = process(
    data['adj_train'], data['features'], normalize_adj, normalize_feats
)


data.keys()

data["adj_train"]

data['adj_train_norm']

data["features"]

data['features']

n_nodes, feat_dim = data['features'].shape

n_nodes, feat_dim

nb_false_edges = len(data['train_edges_false'])
nb_edges = len(data['train_edges'])
nb_false_edges, nb_edges

D:\Nalex\Playgrounds\HyperbolicPackage\branch\hyperlib\hyperlib\utils\data_utils.py:67: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:620.)
  return torch.sparse.FloatTensor(indices, values, shape)


(3664290, 3696)

In [13]:
LinearHyperbolic(data['features'][1].shape, Lorentz(), 1.0, activation=None)

In [14]:
LinearHyperbolic(1433, Lorentz(), 1.0, activation=None)

In [15]:
def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.SparseTensor(indices, coo.data, coo.shape)

## Train Model

In [16]:
hgcn = HGCNLP(1433)

In [17]:
graph.a

<2708x2708 sparse array of type '<class 'numpy.float32'>'
	with 10556 stored elements in Compressed Sparse Row format>

In [18]:
def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.SparseTensor(indices, coo.data, coo.shape)

graph_tf = convert_sparse_matrix_to_sparse_tensor(graph.a)

In [19]:
graph.a

<2708x2708 sparse array of type '<class 'numpy.float32'>'
	with 10556 stored elements in Compressed Sparse Row format>

In [21]:
epochs = 10
for epoch in range(epochs):
    print(f"Epoch {epoch}")
    hgcn((X, graph_tf))
    #grads = tape.gradient(loss_value, self.embedding.trainable_weights)
    #optimizer.apply_gradients(zip(grads, self.embedding.trainable_weights))

    #if step % 100 == 0:
    #    log.info("Training loss (for one batch) at step %d: %.4f"
    #        % (step, float(loss_value)))

Epoch 0
x shape empmap0 (2708, 1433)
HGCLayer x shape (2708, 1433)


InvalidArgumentError: Exception encountered when calling layer 'linear_hyperbolic_3' (type LinearHyperbolic).

{{function_node __wrapped__ConcatV2_N_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} ConcatOp : Ranks of all input tensors should match: shape[0] = [2708,1] vs. shape[1] = [1,2708,1432] [Op:ConcatV2] name: concat

Call arguments received by layer 'linear_hyperbolic_3' (type LinearHyperbolic):
  • inputs=tf.Tensor(shape=(2708, 1433), dtype=float32)

In [ ]:
epochs = 10
for epoch in range(epochs):
    print(f"Epoch {epoch}")
    hgcn((X, graph.a))
    #grads = tape.gradient(loss_value, self.embedding.trainable_weights)
    #optimizer.apply_gradients(zip(grads, self.embedding.trainable_weights))

    #if step % 100 == 0:
    #    log.info("Training loss (for one batch) at step %d: %.4f"
    #        % (step, float(loss_value)))